                                Chapter8 Data Wrangling: Join, Combine and Reshape

# 8.1 Hierarchical indexing 

In [6]:
import pandas as pd

In [7]:
import numpy as np

In [9]:
data = pd.Series(np.random.uniform(size=9), 
                 index = [["a","a","a","b","b","c","c","d","d"],[1,2,3,1,3,1,2,2,3]])

In [10]:
data

a  1    0.660981
   2    0.798556
   3    0.122622
b  1    0.952971
   3    0.906578
c  1    0.908275
   2    0.217713
d  2    0.546109
   3    0.864957
dtype: float64

In [11]:
data.index

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 3),
            ('c', 1),
            ('c', 2),
            ('d', 2),
            ('d', 3)],
           )

In [12]:
data["b"]

1    0.952971
3    0.906578
dtype: float64

In [13]:
data["b":"c"]

b  1    0.952971
   3    0.906578
c  1    0.908275
   2    0.217713
dtype: float64

In [14]:
data.loc[["b","d"]]

b  1    0.952971
   3    0.906578
d  2    0.546109
   3    0.864957
dtype: float64

In [15]:
data

a  1    0.660981
   2    0.798556
   3    0.122622
b  1    0.952971
   3    0.906578
c  1    0.908275
   2    0.217713
d  2    0.546109
   3    0.864957
dtype: float64

In [16]:
data.loc[:,2]

a    0.798556
c    0.217713
d    0.546109
dtype: float64

In [17]:
data.unstack

<bound method Series.unstack of a  1    0.660981
   2    0.798556
   3    0.122622
b  1    0.952971
   3    0.906578
c  1    0.908275
   2    0.217713
d  2    0.546109
   3    0.864957
dtype: float64>

In [18]:
data.unstack()

,1,2,3
a,0.660981,0.798556,0.122622
b,0.952971,NaN,0.906578
c,0.908275,0.217713,NaN
d,NaN,0.546109,0.864957


In [19]:
data.unstack().stack()

a  1    0.660981
   2    0.798556
   3    0.122622
b  1    0.952971
   3    0.906578
c  1    0.908275
   2    0.217713
d  2    0.546109
   3    0.864957
dtype: float64

In [20]:
frame = pd.DataFrame(np.arange(12).reshape((4,3)),
                     index = [["a", "a", "b", "b"], [1,2,1,2]],
                     columns = [["ohio","ohio","colorado"],["green",'red','greed']])

In [21]:
frame

ohio     colorado
    green red    greed
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

In [22]:
frame.index.names = ["key1","key2"]

In [23]:
frame

ohio     colorado
          green red    greed
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [24]:
frame.columns.names = ["state", "color"]

In [25]:
frame

state      ohio     colorado
color     green red    greed
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [27]:
frame.index

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           names=['key1', 'key2'])

## Reordering and Sorting Levels

In [30]:
frame

state      ohio     colorado
color     green red    greed
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [31]:
frame.swaplevel("key1","key2")

state      ohio     colorado
color     green red    greed
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

In [32]:
frame.sort_index(level=1)

state      ohio     colorado
color     green red    greed
key1 key2                   
a    1        0   1        2
b    1        6   7        8
a    2        3   4        5
b    2        9  10       11

In [33]:
frame.swaplevel(0,1).sort_index(level=0)

state      ohio     colorado
color     green red    greed
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11

## Summary Statistics by Level

In [36]:
frame.groupby(level = "key2").sum()

state  ohio     colorado
color green red    greed
key2                    
1         6   8       10
2        12  14       16

In [37]:
frame.groupby(level="color", axis="columns").sum()

color      greed  green  red
key1 key2                   
a    1         2      0    1
     2         5      3    4
b    1         8      6    7
     2        11      9   10

## indexing with a DataFrame's columns

In [40]:
 frame = pd.DataFrame({"a": range(7), "b": range(7, 0, -1),
   ....:                       "c": ["one", "one", "one", "two", "two",
   ....:                             "two", "two"],
   ....:                       "d": [0, 1, 2, 0, 1, 2, 3]})

In [41]:
frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


In [43]:
frame2 = frame.set_index(["c","d"])

In [44]:
frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

In [45]:
frame2.reset_index()

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1


# 8.2 Combining and Merging Datasets

# Database-Style DataFrame Joins

In [63]:
df1 = pd.DataFrame({"key1": ["b","b","a","c","a","a","b"],
                   "data1": pd.Series(range(7),dtype="Int64")})

In [56]:
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [64]:
df2 = pd.DataFrame({"key2": ["a","b","d"],
                   "data2": pd.Series(range(3),dtype="Int64")})

In [60]:
df2

,key,data2
0,a,0
1,b,1
2,d,2


In [61]:
pd.merge(df1,df2)

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [62]:
pd.merge(df1,df2, on="key")

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [65]:
df1

,key1,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [66]:
df2

,key2,data2
0,a,0
1,b,1
2,d,2


In [67]:
pd.merge(df1,df2,left_on="key1",right_on = "key2")

,key1,data1,key2,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0


In [69]:
pd.merge(df1,df2,left_on = "key1", right_on = "key2", how="outer")

,key1,data1,key2,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0
6,c,3,NaN,<NA>
7,NaN,<NA>,d,2


In [70]:
left = pd.DataFrame({
    "key1": ["foo","foo","bar"],
    "key2": ["one","two","one"],
    "lval": pd.Series([1,2,3], dtype="Int64")
})

In [71]:
left

,key1,key2,lval
0,foo,one,1
1,foo,two,2
2,bar,one,3


In [78]:
right = pd.DataFrame({
    "key1": ["foo","foo","bar","bar"],
    "key2": ["one","one","one","two"],
    "rval": pd.Series([4,5,6,7], dtype="Int64")
})

In [76]:
right

,key1,key2,lval
0,foo,one,4
1,foo,one,5
2,bar,one,6
3,bar,two,7


In [79]:
pd.merge(left,right, on=["key1","key2"], how="outer")

,key1,key2,lval,rval
0,foo,one,1,4
1,foo,one,1,5
2,foo,two,2,<NA>
3,bar,one,3,6
4,bar,two,<NA>,7


In [80]:
pd.merge(left, right, on = "key1", suffixes=("_left","_right"))

,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


## Merging on Index

In [87]:
left1 = pd.DataFrame({"key": ["a", "b", "a", "a", "b", "c"],
                    "value": pd.Series(range(6), dtype="Int64")})
right1 = pd.DataFrame({"group_val": [3.5, 7]}, index=["a", "b"])


In [88]:
left1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [89]:
right1

,group_val
a,3.5
b,7.0


In [91]:
pd.merge(left1, right1, left_on="key", right_index=True)

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


In [92]:
pd.merge(left1, right1, left_on="key", right_index=True, how = "outer")

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


In [98]:
lefth = pd.DataFrame({"key1": ["Ohio","Ohio","Ohio","Nevada","Nevada"],
                      "key2": [2000,2001,2002,2001,2002],
                      "data": pd.Series(range(5),dtype="Int64")})

In [99]:
lefth

,key1,key2,data
0,Ohio,2000,0
1,Ohio,2001,1
2,Ohio,2002,2
3,Nevada,2001,3
4,Nevada,2002,4


In [100]:
righth_index = pd.MultiIndex.from_arrays(
[
    ["Nevada","Nevada","Ohio","Ohio","Ohio","Ohio"],
    [2001,2000,2000,2000,2001,2002]
]
)

In [101]:
righth_index

MultiIndex([('Nevada', 2001),
            ('Nevada', 2000),
            (  'Ohio', 2000),
            (  'Ohio', 2000),
            (  'Ohio', 2001),
            (  'Ohio', 2002)],
           )

In [102]:
righth = pd.DataFrame(
{
    "event1": pd.Series([0,2,4,6,8,10], dtype = "Int64", index=righth_index),
    "event2": pd.Series([1,3,5,7,9,11], dtype = "Int64", index=righth_index)
}
)

In [103]:
righth

event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11

In [105]:
pd.merge(lefth,righth,left_on=["key1","key2"], right_index=True)

,key1,key2,data,event1,event2
0,Ohio,2000,0,4,5
0,Ohio,2000,0,6,7
1,Ohio,2001,1,8,9
2,Ohio,2002,2,10,11
3,Nevada,2001,3,0,1


In [108]:
pd.merge(lefth,righth,left_on=["key1","key2"], right_index=True, how="outer")

,key1,key2,data,event1,event2
0,Ohio,2000,0,4,5
0,Ohio,2000,0,6,7
1,Ohio,2001,1,8,9
2,Ohio,2002,2,10,11
3,Nevada,2001,3,0,1
4,Nevada,2002,4,<NA>,<NA>
4,Nevada,2000,<NA>,2,3


In [109]:
In [73]: left2 = pd.DataFrame([[1., 2.], [3., 4.], [5., 6.]],
   ....:                      index=["a", "c", "e"],
   ....:                      columns=["Ohio", "Nevada"]).astype("Int64")

In [74]: right2 = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [13, 14]],
   ....:                       index=["b", "c", "d", "e"],
   ....:                       columns=["Missouri", "Alabama"]).astype("Int64")


In [110]:
left2

,Ohio,Nevada
a,1,2
c,3,4
e,5,6


In [111]:
right2

,Missouri,Alabama
b,7,8
c,9,10
d,11,12
e,13,14


In [113]:
pd.merge(left2, right2, how="outer", left_index=True, right_index=True)

,Ohio,Nevada,Missouri,Alabama
a,1,2,<NA>,<NA>
b,<NA>,<NA>,7,8
c,3,4,9,10
d,<NA>,<NA>,11,12
e,5,6,13,14


In [114]:
left2.join(right2, how="outer")

,Ohio,Nevada,Missouri,Alabama
a,1,2,<NA>,<NA>
b,<NA>,<NA>,7,8
c,3,4,9,10
d,<NA>,<NA>,11,12
e,5,6,13,14


In [115]:
left1.join(right1, on="key")

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0
5,c,5,NaN


In [116]:
another = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [16., 17.]],
   ....:                        index=["a", "c", "e", "f"],
   ....:                        columns=["New York", "Oregon"])

In [81]: another

,New York,Oregon
a,7.0,8.0
c,9.0,10.0
e,11.0,12.0
f,16.0,17.0


In [117]:
left2

,Ohio,Nevada
a,1,2
c,3,4
e,5,6


In [118]:
right2

,Missouri,Alabama
b,7,8
c,9,10
d,11,12
e,13,14


In [119]:
left2.join([right2, another])

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1,2,<NA>,<NA>,7.0,8.0
c,3,4,9,10,9.0,10.0
e,5,6,13,14,11.0,12.0


In [122]:
left2.join([right2,another], how="outer")

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1,2,<NA>,<NA>,7.0,8.0
c,3,4,9,10,9.0,10.0
e,5,6,13,14,11.0,12.0
b,<NA>,<NA>,7,8,NaN,NaN
d,<NA>,<NA>,11,12,NaN,NaN
f,<NA>,<NA>,<NA>,<NA>,16.0,17.0


## Concatenating Along an Axis

In [125]:
arr = np.arange(12).reshape((3,4))

In [126]:
arr

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [127]:
np.concatenate([arr,arr], axis=1)

array([[ 0,  1,  2,  3,  0,  1,  2,  3],
       [ 4,  5,  6,  7,  4,  5,  6,  7],
       [ 8,  9, 10, 11,  8,  9, 10, 11]])

In [132]:
s1 = pd.Series([0,1], index=["a","b"], dtype="Int64")

In [133]:
s2 = pd.Series([2,3,4], index=["c","d","e"], dtype="Int64")

In [134]:
s3 = pd.Series([5,6], index=["f","g"], dtype="Int64")

In [135]:
s1

a    0
b    1
dtype: Int64

In [136]:
s2

c    2
d    3
e    4
dtype: Int64

In [137]:
s3

f    5
g    6
dtype: Int64

In [138]:
pd.concat([s1,s2,s3])

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: Int64

In [139]:
pd.concat([s1,s2,s3], axis="columns")

,0,1,2
a,0,<NA>,<NA>
b,1,<NA>,<NA>
c,<NA>,2,<NA>
d,<NA>,3,<NA>
e,<NA>,4,<NA>
f,<NA>,<NA>,5
g,<NA>,<NA>,6


In [140]:
s4 = pd.concat([s1, s3])

In [141]:
s4

a    0
b    1
f    5
g    6
dtype: Int64

In [142]:
pd.concat([s1,s4], axis="columns")

,0,1
a,0,0
b,1,1
f,<NA>,5
g,<NA>,6


In [143]:
pd.concat([s1,s4], axis="columns", join="inner")

,0,1
a,0,0
b,1,1


In [144]:
result = pd.concat([s1,s1,s3],keys=["one","two","three"])

In [145]:
result

one    a    0
       b    1
two    a    0
       b    1
three  f    5
       g    6
dtype: Int64

In [146]:
result.unstack()

,a,b,f,g
one,0,1,<NA>,<NA>
two,0,1,<NA>,<NA>
three,<NA>,<NA>,5,6


In [151]:
pd.concat([s1, s2, s3], axis="columns", keys=["one", "two", "three"])

,one,two,three
a,0,<NA>,<NA>
b,1,<NA>,<NA>
c,<NA>,2,<NA>
d,<NA>,3,<NA>
e,<NA>,4,<NA>
f,<NA>,<NA>,5
g,<NA>,<NA>,6


In [152]:
result

,one,two,three
a,0,0,<NA>
b,1,1,<NA>
f,<NA>,<NA>,5
g,<NA>,<NA>,6


In [153]:
df1 = pd.DataFrame(np.arange(6).reshape(3,2), index=["a","b","c"], columns=["one","two"])

In [154]:
df1

,one,two
a,0,1
b,2,3
c,4,5


In [155]:
df2 = pd.DataFrame(5 + np.arange(4).reshape(2,2), index=["a","c"], columns=["three","four"])

In [156]:
df2

,three,four
a,5,6
c,7,8


In [157]:
pd.concat([df1,df2],axis="columns",keys=["level1","level2"])

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

In [158]:
pd.concat({"level1":df1, "level2":df2}, axis= "columns")

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

In [159]:
pd.concat({"level1":df1, "level2":df2}, axis= "columns", names=["upper","lower"])

upper level1     level2     
lower    one two  three four
a          0   1    5.0  6.0
b          2   3    NaN  NaN
c          4   5    7.0  8.0

In [160]:
In [110]: df1 = pd.DataFrame(np.random.standard_normal((3, 4)),
   .....:                    columns=["a", "b", "c", "d"])

In [111]: df2 = pd.DataFrame(np.random.standard_normal((2, 3)),
   .....:                    columns=["b", "d", "a"])


In [161]:
df1

,a,b,c,d
0,-0.462370,0.211104,1.644474,-1.428259
1,1.045819,-1.248845,-0.945111,-1.046795
2,1.513220,0.399787,2.008753,-0.460323


In [162]:
df2

,b,d,a
0,1.510944,-2.202244,-0.316264
1,-0.260579,0.619742,-0.586237


In [163]:
pd.concat([df1, df2], ignore_index=True)

,a,b,c,d
0,-0.462370,0.211104,1.644474,-1.428259
1,1.045819,-1.248845,-0.945111,-1.046795
2,1.513220,0.399787,2.008753,-0.460323
3,-0.316264,1.510944,NaN,-2.202244
4,-0.586237,-0.260579,NaN,0.619742


## Combining Data with Overlap

In [166]:
In [115]: a = pd.Series([np.nan, 2.5, 0.0, 3.5, 4.5, np.nan],
   .....:               index=["f", "e", "d", "c", "b", "a"])

In [116]: b = pd.Series([0., np.nan, 2., np.nan, np.nan, 5.],
   .....:               index=["a", "b", "c", "d", "e", "f"])


In [167]:
a

f    NaN
e    2.5
d    0.0
c    3.5
b    4.5
a    NaN
dtype: float64

In [168]:
b

a    0.0
b    NaN
c    2.0
d    NaN
e    NaN
f    5.0
dtype: float64

In [169]:
np.where(pd.isna(a), b, a)

array([0. , 2.5, 0. , 3.5, 4.5, 5. ])

In [170]:
a.combine_first(b)

a    0.0
b    4.5
c    3.5
d    0.0
e    2.5
f    5.0
dtype: float64

In [171]:
In [121]: df1 = pd.DataFrame({"a": [1., np.nan, 5., np.nan],
   .....:                     "b": [np.nan, 2., np.nan, 6.],
   .....:                     "c": range(2, 18, 4)})

In [122]: df2 = pd.DataFrame({"a": [5., 4., np.nan, 3., 7.],
   .....:                     "b": [np.nan, 3., 4., 6., 8.]})

In [172]:
df1

,a,b,c
0,1.0,NaN,2
1,NaN,2.0,6
2,5.0,NaN,10
3,NaN,6.0,14


In [173]:
df2

,a,b
0,5.0,NaN
1,4.0,3.0
2,NaN,4.0
3,3.0,6.0
4,7.0,8.0


In [174]:
df1.combine_first(df2)

,a,b,c
0,1.0,NaN,2.0
1,4.0,2.0,6.0
2,5.0,4.0,10.0
3,3.0,6.0,14.0
4,7.0,8.0,NaN


# 8.3 Reshape and Pivoting 

## Reshaping with Hierarchical Indexing

In [178]:
In [126]: data = pd.DataFrame(np.arange(6).reshape((2, 3)),
   .....:                     index=pd.Index(["Ohio", "Colorado"], name="state"),
   .....:                     columns=pd.Index(["one", "two", "three"],
   .....:                     name="number"))

In [179]:
data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [180]:
result = data.stack()

In [181]:
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int64

In [182]:
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int64

In [183]:
result.unstack()

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [184]:
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int64

In [185]:
result.unstack(level=0)

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [186]:
result.unstack(level="state")

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [189]:
s1 = pd.Series([0,1,2,3], index=["a","b","c","d"], dtype="Int64")

In [190]:
s2 = pd.Series([4,5,6], index=["c","d","e"], dtype="Int64")

In [191]:
s1

a    0
b    1
c    2
d    3
dtype: Int64

In [192]:
s2

c    4
d    5
e    6
dtype: Int64

In [193]:
data2 = pd.concat([s1,s2], keys=["one","two"])

In [194]:
data2

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: Int64

In [195]:
data2.unstack()

,a,b,c,d,e
one,0,1,2,3,<NA>
two,<NA>,<NA>,4,5,6


In [196]:
data2.unstack().stack()

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: Int64

In [197]:
data2.unstack().stack(dropna=False)

one  a       0
     b       1
     c       2
     d       3
     e    <NA>
two  a    <NA>
     b    <NA>
     c       4
     d       5
     e       6
dtype: Int64

In [198]:
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int64

In [203]:
df = pd.DataFrame({"left": result, "right": result+5}, columns = pd.Index(["left","right"], name="side"))

In [204]:
df

side             left  right
state    number             
Ohio     one        0      5
         two        1      6
         three      2      7
Colorado one        3      8
         two        4      9
         three      5     10

In [205]:
df.unstack(level="state")

side   left          right         
state  Ohio Colorado  Ohio Colorado
number                             
one       0        3     5        8
two       1        4     6        9
three     2        5     7       10

In [206]:
df.unstack(level="state").stack(level="side")

state         Colorado  Ohio
number side                 
one    left          3     0
       right         8     5
two    left          4     1
       right         9     6
three  left          5     2
       right        10     7

## Pivoting "Long" to "Wide" Format

In [209]:
data = pd.read_csv("/Users/yuxichen/online eduction/python_data_analysis/pydata-book-3rd-edition/examples/macrodata.csv")

In [210]:
data

,year,quarter,realgdp,realcons,realinv,realgovt,realdpi,cpi,m1,tbilrate,unemp,pop,infl,realint
0,1959,1,2710.349,1707.4,286.898,470.045,1886.9,28.980,139.7,2.82,5.8,177.146,0.00,0.00
1,1959,2,2778.801,1733.7,310.859,481.301,1919.7,29.150,141.7,3.08,5.1,177.830,2.34,0.74
2,1959,3,2775.488,1751.8,289.226,491.260,1916.4,29.350,140.5,3.82,5.3,178.657,2.74,1.09
3,1959,4,2785.204,1753.7,299.356,484.052,1931.3,29.370,140.0,4.33,5.6,179.386,0.27,4.06
4,1960,1,2847.699,1770.5,331.722,462.199,1955.5,29.540,139.6,3.50,5.2,180.007,2.31,1.19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198,2008,3,13324.600,9267.7,1990.693,991.551,9838.3,216.889,1474.7,1.17,6.0,305.270,-3.16,4.33
199,2008,4,13141.920,9195.3,1857.661,1007.273,9920.4,212.174,1576.5,0.12,6.9,305.952,-8.79,8.91
200,2009,1,12925.410,9209.2,1558.494,996.287,9926.4,212.671,1592.8,0.22,8.1,306.547,0.94,-0.71
201,2009,2,12901.504,9189.0,1456.678,1023.528,10077.5,214.469,1653.6,0.18,9.2,307.226,3.37,-3.19


In [211]:
data = data.loc[:, ["year", "quarter", "realgdp", "infl", "unemp"]]

In [212]:
data

,year,quarter,realgdp,infl,unemp
0,1959,1,2710.349,0.00,5.8
1,1959,2,2778.801,2.34,5.1
2,1959,3,2775.488,2.74,5.3
3,1959,4,2785.204,0.27,5.6
4,1960,1,2847.699,2.31,5.2
...,...,...,...,...,...
198,2008,3,13324.600,-3.16,6.0
199,2008,4,13141.920,-8.79,6.9
200,2009,1,12925.410,0.94,8.1
201,2009,2,12901.504,3.37,9.2


In [213]:
periods = pd.PeriodIndex(year=data.pop("year"), quarter=data.pop("quarter"), name="date")

In [214]:
periods

PeriodIndex(['1959Q1', '1959Q2', '1959Q3', '1959Q4', '1960Q1', '1960Q2',
             '1960Q3', '1960Q4', '1961Q1', '1961Q2',
             ...
             '2007Q2', '2007Q3', '2007Q4', '2008Q1', '2008Q2', '2008Q3',
             '2008Q4', '2009Q1', '2009Q2', '2009Q3'],
            dtype='period[Q-DEC]', name='date', length=203)

In [215]:
data.index = periods.to_timestamp("D")

In [216]:
data.head()

,realgdp,infl,unemp
date,,,
1959-01-01,2710.349,0.00,5.8
1959-04-01,2778.801,2.34,5.1
1959-07-01,2775.488,2.74,5.3
1959-10-01,2785.204,0.27,5.6
1960-01-01,2847.699,2.31,5.2


In [217]:
data

,realgdp,infl,unemp
date,,,
1959-01-01,2710.349,0.00,5.8
1959-04-01,2778.801,2.34,5.1
1959-07-01,2775.488,2.74,5.3
1959-10-01,2785.204,0.27,5.6
1960-01-01,2847.699,2.31,5.2
...,...,...,...
2008-07-01,13324.600,-3.16,6.0
2008-10-01,13141.920,-8.79,6.9
2009-01-01,12925.410,0.94,8.1


In [218]:
data.columns.name = "item"

In [219]:
data.head()

item,realgdp,infl,unemp
date,,,
1959-01-01,2710.349,0.00,5.8
1959-04-01,2778.801,2.34,5.1
1959-07-01,2775.488,2.74,5.3
1959-10-01,2785.204,0.27,5.6
1960-01-01,2847.699,2.31,5.2


In [221]:
long_data = (data.stack().reset_index().rename(columns={0:"values"}))

In [222]:
long_data

,date,item,values
0,1959-01-01,realgdp,2710.349
1,1959-01-01,infl,0.000
2,1959-01-01,unemp,5.800
3,1959-04-01,realgdp,2778.801
4,1959-04-01,infl,2.340
...,...,...,...
604,2009-04-01,infl,3.370
605,2009-04-01,unemp,9.200
606,2009-07-01,realgdp,12990.341
607,2009-07-01,infl,3.560


In [226]:
pivoted = long_data.pivot(index="date", columns="item",
                         values="values")

In [227]:
pivoted

item,infl,realgdp,unemp
date,,,
1959-01-01,0.00,2710.349,5.8
1959-04-01,2.34,2778.801,5.1
1959-07-01,2.74,2775.488,5.3
1959-10-01,0.27,2785.204,5.6
1960-01-01,2.31,2847.699,5.2
...,...,...,...
2008-07-01,-3.16,13324.600,6.0
2008-10-01,-8.79,13141.920,6.9
2009-01-01,0.94,12925.410,8.1


In [228]:
long_data["value2"] = np.random.standard_normal(len(long_data))

In [229]:
long_data

,date,item,values,value2
0,1959-01-01,realgdp,2710.349,-1.648777
1,1959-01-01,infl,0.000,-1.669199
2,1959-01-01,unemp,5.800,0.501523
3,1959-04-01,realgdp,2778.801,-1.115909
4,1959-04-01,infl,2.340,-0.124962
...,...,...,...,...
604,2009-04-01,infl,3.370,-0.720792
605,2009-04-01,unemp,9.200,0.043774
606,2009-07-01,realgdp,12990.341,0.622344
607,2009-07-01,infl,3.560,0.926340


In [230]:
pivoted = long_data.pivot(index="date",columns="item")

In [231]:
pivoted.head()

values                    value2                    
item         infl   realgdp unemp      infl   realgdp     unemp
date                                                           
1959-01-01   0.00  2710.349   5.8 -1.669199 -1.648777  0.501523
1959-04-01   2.34  2778.801   5.1 -0.124962 -1.115909 -1.237075
1959-07-01   2.74  2775.488   5.3 -0.749027 -0.138601  2.023772
1959-10-01   0.27  2785.204   5.6  0.912099  0.313214 -0.224166
1960-01-01   2.31  2847.699   5.2 -0.698323  0.540977  1.078189

In [233]:
long_data.set_index(["date","item"]).unstack(level="item")

values                     value2                    
item         infl    realgdp unemp      infl   realgdp     unemp
date                                                            
1959-01-01   0.00   2710.349   5.8 -1.669199 -1.648777  0.501523
1959-04-01   2.34   2778.801   5.1 -0.124962 -1.115909 -1.237075
1959-07-01   2.74   2775.488   5.3 -0.749027 -0.138601  2.023772
1959-10-01   0.27   2785.204   5.6  0.912099  0.313214 -0.224166
1960-01-01   2.31   2847.699   5.2 -0.698323  0.540977  1.078189
...           ...        ...   ...       ...       ...       ...
2008-07-01  -3.16  13324.600   6.0  1.094852  2.152015  2.479598
2008-10-01  -8.79  13141.920   6.9  1.388794  1.065943  0.492242
2009-01-01   0.94  12925.410   8.1  0.434121  0.949365  0.224604
2009-04-01   3.37  12901.504   9.2 -0.720792  0.035577  0.043774
2009-07-01   3.56  12990.341   9.6  0.926340  0.622344 -0.804521

[203 rows x 6 columns]

## Pivoting Wide to Long Format

In [236]:
df = pd.DataFrame({"key": ["foo", "bar", "baz"],
   .....:                    "A": [1, 2, 3],
   .....:                    "B": [4, 5, 6],
   .....:                    "C": [7, 8, 9]})


In [237]:
df

,key,A,B,C
0,foo,1,4,7
1,bar,2,5,8
2,baz,3,6,9


In [238]:
melted = pd.melt(df, id_vars="key")

In [239]:
melted

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6
6,foo,C,7
7,bar,C,8
8,baz,C,9


In [243]:
reshaped = melted.pivot(index="key", columns="variable", values="value")

In [244]:
reshaped

variable,A,B,C
key,,,
bar,2,5,8
baz,3,6,9
foo,1,4,7


In [245]:
reshaped.reset_index()

variable,key,A,B,C
0,bar,2,5,8
1,baz,3,6,9
2,foo,1,4,7


In [246]:
pd.melt(df, id_vars="key", value_vars=["A","B"])

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6
